<div style='width: auto; float: center'/>
<table align="center">
    <tr>
        <th>
                <img src="https://ingenieria.unal.edu.co/pri/images/catedra_header_2021.png" style='height: 200px; width: auto; object-fit: contain;float: right'/>                                		
        </th>     
    </tr>
</table>

</div>

<div >
  <h1 align="center">  Asset Management Principles For Modern Power Systems    </h1>  
  <h2 align="center">  Topic 1: Crticality assessment     </h2>      
			<h3 align="center">    David L. Alvarez A, Ph.D    </h3>
			<h4 align="center">    dlalvareza@unal.edu.co      </h4>
            <h4 align="center">    Copyright (c) 2021 dlalvareza    </h4>
</div>	

<div style='width: auto; float: center'/>
<table align="center">
    <tr>
        <th>
       <img src="https://www.uqtr.ca/u20-img/logos/logo-uqtr-entete.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                	
        </th>                 
        <th>
                <img src="https://oraprdnt.uqtr.uquebec.ca/pls/public/docs/GSC6627/O0003430433_GREI_LOGO_court.PNG" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>  
        <th>
                <img src="https://oraprdnt.uqtr.uquebec.ca/pls/public/docs/GSC6627/O0003430597_MRIF_w3_couleur.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>  
        <th>
       <img src="https://lh3.googleusercontent.com/IRmEQuk5greQjkXJHTjwSVNgc58Kd56aiP5OCWN3mlAJUYd_AHUM4j7vn05grSKZL-ABbQ=s170" style='height: 50px; width: auto; object-fit: contain;float: right'/>                                	
        </th>                 
        <th>
                <img src="https://unal.edu.co/typo3conf/ext/unal_skin_default/Resources/Public/images/escudoUnal_black.png" style='height: 75px; width: auto; object-fit: contain;float: right'/>                                		
        </th>          
    </tr>
</table>    

## CIGRE network benchmark DER in Medium Voltage Systems
![title](../STATIC/01_CIGRE_MV_Distribution_DER_Assets_Criticality.svg)

## Cost of replacing assets
| Asset Type| Cost |
| --- | --- | 
| TR  | 2e4 | 
| OHL | 2.25e4 |
| CB  | 5e3|
| AUX | 2e3 |

### Neccesary libraries

In [1]:
import sys
import datetime 
import pandas as pd
import calendar
from ipywidgets import interact
from ipywidgets import fixed
import plotly.express as px
from plotly.subplots import make_subplots

### Load PywerAPM libraries

In [2]:
sys.path.insert(0,'../CASES/05_Challenge_Data/')
sys.path.insert(0,'../APM/BIN/')
from ST_AM_Contingencies_Analysis import Real_Time_Contingencies as Cont_Assessment
from ST_AM_Contingencies_Ploty import  Plot_Stack_By_Day, Plot_Security_Opteration

# 1. Normal opertaion 

## 1.1 Import case settings

In [3]:
from PywerAPM_Case_Setting import*

## 1.2 Create contingencies assessment object

In [4]:
Cont_A = Cont_Assessment(net_file,load_data,pp_case='json')

## 1.3 Upade load growth

In [5]:
Cont_A.Load_Growth_Update(load_growth)

## 1.4 Assessment of load durign a week

In [6]:
df_base_line,df_load,_ = Cont_A.Run_Case_Load_growth(Cont_A.net,load_growth)
S_base = sqrt((Cont_A.net.load['p_mw']**2+  Cont_A.net.load['q_mvar']**2).max())

## 1.5 Plot load profiles by week day during 24 h

In [7]:
def plot_load_profile(day_name):
    fig = Plot_Stack_By_Day(df_load,day_name)

In [8]:
day_list = list(calendar.day_name)
interact(plot_load_profile, day_name=day_list)

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

<function __main__.plot_load_profile(day_name)>

## 1.6 Plot security assessment by week day during 24 h

In [9]:
def plot_security_profile_type(day_name,df):
    l_asset = ['TR','BU','LN']
    interact(plot_security_profile,l_type=l_asset,day_name=fixed(day_name),df = fixed(df))
    
def plot_security_profile(l_type,day_name,df):
    TR = False
    LN = False
    BU = False
    if l_type=='TR':
        TR  = True
    elif l_type=='LN':
        LN  = True
    elif l_type=='BU':
        BU  = True
    Plot_Security_Opteration(df,day_name,S_base,TR,LN,BU)

In [10]:
interact(plot_security_profile_type, day_name=day_list,df=fixed(df_base_line))

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

<function __main__.plot_security_profile_type(day_name, df)>

# 2. Contingency assessment

In [11]:
def Contingency_Assessment(day_name=day_list):
    asset_type_list = Cont_A.asset_list['Type'].unique()
    interact(Asset_Selection, asset_type=asset_type_list,day=fixed(day_name))    
def Asset_Selection(asset_type,day):
    asset_id_list                = Cont_A.asset_list[Cont_A.asset_list['Type']==asset_type].index.values
    interact(disc_asset, asset_id=asset_id_list,day=fixed(day))  
def disc_asset(asset_id,day):
    asset_fail                        = asset_id
    net_lf                            = Cont_A.Net_Configurarion_during_Contingency(Cont_A.net,asset_fail)
    df_base_C,df_load_C,sec_assesment = Cont_A.Run_Case_Load_growth(net_lf,load_growth,day=[day])
    plot_security_profile_type(day,df_base_C)
    hour_list = df_base_C[df_base_C.Day==day].Hour.unique()
    sec_assesment_by_day = sec_assesment['cont'][day]
    interact(df_security_violation_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day))  
def df_security_violation_by_hour(hour,df_sec):
    df = df_sec[hour]
    if df.empty:
        print('None security violations')
    else:
        print(df)

## 2.1 Plot security assessment by week day for an asset disconection during 24 h

In [12]:
interact(Contingency_Assessment, day_name=day_list)

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

<function __main__.Contingency_Assessment(day_name=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])>

## 2.2 Energy supplied and generation assessment

In [13]:
def Contingency_Assessment_Energy(day_name=day_list):
    asset_type_list = Cont_A.asset_list['Type'].unique()
    interact(Asset_Selection_E, asset_type=asset_type_list,day=fixed(day_name))    
def Asset_Selection_E(asset_type,day):
    asset_id_list                = Cont_A.asset_list[Cont_A.asset_list['Type']==asset_type].index.values
    interact(disc_asset_E, asset_id=asset_id_list,day=fixed(day))  
def disc_asset_E(asset_id,day):
    asset_fail                        = asset_id
    net_lf                            = Cont_A.Net_Configurarion_during_Contingency(Cont_A.net,asset_fail)
    df_base_C,df_load_C,sec_assesment = Cont_A.Run_Case_Load_growth(net_lf,load_growth,day=[day])
    hour_list = df_base_C[df_base_C.Day==day].Hour.unique()
    sec_assesment_by_day =  sec_assesment['cr_energy'][day]
    interact(Energy_Assessment_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day))  
def Energy_Assessment_by_hour(hour,df_sec):
    # Load
    df      = df_sec[hour]
    df_load = df['Load']
    df_load = df_load.melt(id_vars =['name']) 
    fig1    = px.sunburst(df_load, path=['variable','name'], values='value')
    # Generation
    df_gen = df['Gen']
    fig2 = px.sunburst(df_gen, path=['source','name'], values='gen')
    # ploting    
    fig = make_subplots(rows=1, cols=2, specs=[[{"type": "sunburst"}, {"type": "sunburst"}]])
    fig.add_trace(fig1.data[0], row=1, col=1)
    fig.add_trace(fig2.data[0], row=1, col=2)
    fig.show()

In [14]:
interact(Contingency_Assessment_Energy, day_name=day_list)

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

<function __main__.Contingency_Assessment_Energy(day_name=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])>

## 2.3 Operational Criticality  monetizacion 
$$Benefit = ES_c-EG_c-ENS_c$$

In [15]:
from ACM_Module import ACM
criticality = ACM(Cont_A.load_user,Cont_A.gen_data)

In [16]:
def Contingency_Assessment_Cost(day_name=day_list):
    asset_type_list = Cont_A.asset_list['Type'].unique()
    interact(Asset_Selection_Cost, asset_type=asset_type_list,day=fixed(day_name))    
def Asset_Selection_Cost(asset_type,day):
    asset_id_list                = Cont_A.asset_list[Cont_A.asset_list['Type']==asset_type].index.values
    interact(disc_asset_Cost, asset_id=asset_id_list,day=fixed(day))  
def disc_asset_Cost(asset_id,day):
    asset_fail                        = asset_id
    net_lf                            = Cont_A.Net_Configurarion_during_Contingency(Cont_A.net,asset_fail)
    df_base_C,df_load_C,sec_assesment = Cont_A.Run_Case_Load_growth(net_lf,load_growth,day=[day])
    hour_list = df_base_C[df_base_C.Day==day].Hour.unique()
    sec_assesment_by_day =  sec_assesment['cr_energy'][day]
    interact(Cost_Assessment_by_hour, hour=hour_list,df_sec=fixed(sec_assesment_by_day))  
def Cost_Assessment_by_hour(hour,df_sec):
    # Load
    df           = df_sec[hour]
    df_load      = df['Load']
    df_cr_load   = criticality.Monetized_Energy_During_Contingency_by_hour(df_load) 
    df_cr_load   = df_cr_load.reset_index()
    df_load      = df_cr_load.melt(id_vars =['name']) 
    fig1         = px.sunburst(df_load, path=['variable','name'], values='value')
    # Generation
    df_gen       = df['Gen']
    df_cr_gen    = criticality.Monetized_Gen_During_Contingency_by_hour(df_gen) 
    df_cr_gen    = df_cr_gen.reset_index()
    df_gen       = df_cr_gen.melt(id_vars =['name']) 
    fig2         = px.sunburst(df_gen, path=['variable','name'], values='value')
    # ploting    
    fig          = make_subplots(rows=1, cols=2, specs=[[{"type": "sunburst"}, {"type": "sunburst"}]])
    fig.add_trace(fig1.data[0], row=1, col=1)
    fig.add_trace(fig2.data[0], row=1, col=2)
    fig.show()
    benefit      =  str(df_cr_load['ES_Ben'].sum()-df_cr_load['ENS_Cost'].sum()-df_cr_gen['Gen_Cost'].sum()-df_cr_gen['Gen_Env'].sum())
    print('The benfit is ='+ benefit)

In [17]:
interact(Contingency_Assessment_Cost, day_name=day_list)

interactive(children=(Dropdown(description='day_name', options=('Monday', 'Tuesday', 'Wednesday', 'Thursday', …

<function __main__.Contingency_Assessment_Cost(day_name=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'])>